## Importing Required Libraries

In [42]:
import pandas as pd
import openai
import ast

## Enabling GenAI

In [43]:
key = "sk-proj-T0QH2ygkWcqf3efA9ck6T3BlbkFJkVvGVl5vFUUB7ZDlxCbH"
openai.api_key = key
openai.models.list()

SyncPage[Model](data=[Model(id='gpt-3.5-turbo-16k-0613', created=1685474247, object='model', owned_by='openai'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'), Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='mode

In [44]:
from openai import OpenAI
client = OpenAI(api_key = key)

## Importing Data from the TXT and converting it to List

In [45]:
with open('Food Data.txt', 'r', encoding='utf-8', errors='ignore') as file:
    lines = file.readlines()
    literals = []

    for line in lines:
        literals.append(ast.literal_eval(f'"{line.strip()}"'))

literals

["Hi, I'm Ravi. I love Pizza and Burger of LaPinoz in Jaipur. I spent a whopping Rs. 9000 on food last month!",
 "Hey there, it's Sameer from Delhi. Butter Chicken and Naan from Zaika are my go-to comfort food. I try to limit myself to Rs. 5000 a month on takeout. 🫓",
 "Namaste! I'm Kiran, a big fan of the Chole Bhature at Chandni Chowk in Delhi. So delicious and affordable! I usually spend around Rs. 2000 a month on eating out.",
 "What's up? This is Maya from Bengaluru. Can't wait to try the Rajma Chawal at Biryani Paradise tonight. I budgeted Rs. 3000 for eating out this month, but it might not be enough!",
 "Hi everyone, it's Akash from Chennai. Huge fan of the Thali meals at Saravana Bhavan. So much variety! They're a steal at Rs. 250 per meal, so I can eat there often and stay within my Rs. 4000 budget.",
 "Hey! I'm Neha from Mumbai. Love the Aloo Gobi and Roti at Ghar كي (Ki) Rasoi. Feels like home! It's a bit pricey, but worth it. I spend around Rs. 6000 a month on eating out."

## Function Calling Tool

In [46]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_customer_info",
                "description": "Get the customer information from the text",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "Name of the customer"
                        },
                        "fav_food": {
                            "type": "string",
                            "description": "Favorite food ordered"
                        },
                        "fav_restaurant": {
                            "type": "string",
                            "description": "Favorite restaurant of customer"
                        },
                        "total_amt": {
                            "type": "string",
                            "description": "Total amount spent"
                        },
                        "city": {
                            "type": "string",
                            "description": "City customer belongs to"
                        }
                    }
                }
            }
        }
    ]

## Converting the list to individual dictionaries and appending into one

In [47]:
dfs = []

for s in literals:
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {
          "role": "user",
          "content": s
        }
      ],
        tools=tools
    )

    res = response.choices[0].message.tool_calls[0].function.arguments
    my_dict = ast.literal_eval(res)
    my_dict
    dfs.append(my_dict)

In [48]:
dfs

[{'name': 'Ravi',
  'fav_food': 'Pizza, Burger',
  'fav_restaurant': 'LaPinoz',
  'total_amt': 'Rs. 9000',
  'city': 'Jaipur'},
 {'name': 'Sameer',
  'fav_food': 'Butter Chicken and Naan',
  'fav_restaurant': 'Zaika',
  'total_amt': 'Rs. 5000',
  'city': 'Delhi'},
 {'name': 'Kiran',
  'fav_food': 'Chole Bhature',
  'fav_restaurant': 'Chandni Chowk',
  'total_amt': 'Rs. 2000',
  'city': 'Delhi'},
 {'name': 'Maya',
  'fav_food': 'Rajma Chawal',
  'fav_restaurant': 'Biryani Paradise',
  'total_amt': 'Rs. 3000',
  'city': 'Bengaluru'},
 {'name': 'Akash',
  'fav_food': 'Thali meals',
  'fav_restaurant': 'Saravana Bhavan',
  'total_amt': 'Rs. 4000',
  'city': 'Chennai'},
 {'name': 'Neha',
  'fav_food': 'Aloo Gobi and Roti',
  'fav_restaurant': 'Ghar كي Rasoi',
  'total_amt': 'Rs. 6000',
  'city': 'Mumbai'},
 {'name': 'Rohan',
  'fav_food': 'Biryani',
  'fav_restaurant': 'Paradise Biryani',
  'total_amt': 'Rs. 4500',
  'city': 'Hyderabad'},
 {'name': 'Aisha',
  'fav_food': 'Pav Bhaji',
  'fav

In [49]:
final_df = pd.DataFrame(dfs)
final_df

,name,fav_food,fav_restaurant,total_amt,city
0,Ravi,"Pizza, Burger",LaPinoz,Rs. 9000,Jaipur
1,Sameer,Butter Chicken and Naan,Zaika,Rs. 5000,Delhi
2,Kiran,Chole Bhature,Chandni Chowk,Rs. 2000,Delhi
3,Maya,Rajma Chawal,Biryani Paradise,Rs. 3000,Bengaluru
4,Akash,Thali meals,Saravana Bhavan,Rs. 4000,Chennai
5,Neha,Aloo Gobi and Roti,Ghar كي Rasoi,Rs. 6000,Mumbai
6,Rohan,Biryani,Paradise Biryani,Rs. 4500,Hyderabad
7,Aisha,Pav Bhaji,Aadyaa,Rs. 3500,Pune
8,Dev,Dosa and Uttapam,Sankalp,Rs. 2000,Chennai
9,Mira,Masala Dosa,Iyengar Bakery,Rs. 2500,Bengaluru


In [50]:
def get_restaurant_suggestion(fav_food, city):
    try:
        # Make a request to the OpenAI API to get restaurant suggestions
        response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
        {
          "role": "user",
          "content": f'''Can you suggest 1 top restaurant which serves the best {fav_food} in {city}.
          Can you please answer with just the restaurant name and nothing else''',
        }
      ],
        max_tokens=30
    )
        # Check if the API response is valid
        if response.choices and response.choices[0].message and response.choices[0].message.content:
            suggestion = response.choices[0].message.content
            suggested_restaurant = suggestion.split(':')[0].strip()
            return suggested_restaurant
        else:
            return 'No suggestion found'
    except Exception as e:
        print(f"Error fetching suggestion: {e}")
        return 'Error fetching suggestion'

# Create an empty list to store restaurant names
suggested_restaurant = []

# Iterate over each row in the DataFrame using a for loop
for index, row in final_df.iterrows():
    fav_food = row['fav_food']
    city = row['city']
    name = get_restaurant_suggestion(fav_food, city)
    suggested_restaurant.append(name)

# Add the restaurant names as a new column in the DataFrame
final_df['suggested_restaurant'] = suggested_restaurant

In [51]:
final_df

,name,fav_food,fav_restaurant,total_amt,city,suggested_restaurant
0,Ravi,"Pizza, Burger",LaPinoz,Rs. 9000,Jaipur,Blackout
1,Sameer,Butter Chicken and Naan,Zaika,Rs. 5000,Delhi,Punjab Grill
2,Kiran,Chole Bhature,Chandni Chowk,Rs. 2000,Delhi,Sita Ram Diwan Chand
3,Maya,Rajma Chawal,Biryani Paradise,Rs. 3000,Bengaluru,Punjab Grill
4,Akash,Thali meals,Saravana Bhavan,Rs. 4000,Chennai,Rasavid Buffet
5,Neha,Aloo Gobi and Roti,Ghar كي Rasoi,Rs. 6000,Mumbai,Aaswad
6,Rohan,Biryani,Paradise Biryani,Rs. 4500,Hyderabad,Paradise Biryani
7,Aisha,Pav Bhaji,Aadyaa,Rs. 3500,Pune,Sujata Mastani
8,Dev,Dosa and Uttapam,Sankalp,Rs. 2000,Chennai,Saravana Bhavan
9,Mira,Masala Dosa,Iyengar Bakery,Rs. 2500,Bengaluru,CTR
